In [1]:
## Trim down imports to only neccesary 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from transformers import AutoFeatureExtractor, ViTForImageClassification, ViTModel
import torch
from torchvision.transforms import v2
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('default')
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.utils.data import Subset
from collections import namedtuple
from sklearn.preprocessing import StandardScaler
import matplotlib.gridspec as gridspec

import os
import datetime
import re
import sys

## Preprocessing loc
preprocess_dir = '/Users/willhoff/Desktop/thesis_2024/preprocessing'
sys.path.append(preprocess_dir)
from simple_datasets import load_data



/Users/willhoff/Desktop/research_23_24/research_09_11/env/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/willhoff/Desktop/research_23_24/research_09_11/env/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still cha

In [2]:
## Pretrained model

class CustomViTEmbeddingModel(torch.nn.Module):
    def __init__(self, original_model):
        super(CustomViTEmbeddingModel, self).__init__()
        
        # Extract the necessary layers from the original model
        self.embeddings = original_model.vit.embeddings  #.patch_embeddings
        self.encoder_layer_0 = original_model.vit.encoder.layer[0]
        self.encoder_layer_1 = original_model.vit.encoder.layer[1]
        
        # Assume a square grid of patches to reshape the sequence of patches back into a 2D grid
            ## image: 224x224 ; patch size: 16x16 --> 14x14 
        self.num_patches_side = 14

    def forward(self, x):
        # Apply the embeddings layer
        x = self.embeddings(x)
        
        # Pass the result through the first and second encoder layers
        x = self.encoder_layer_0(x)[0]  # [0] to get the hidden states
        x = self.encoder_layer_1(x)[0]  # [0] to get the hidden states
        
        # x is now the sequence of embeddings for the patches
            # The output x will be a sequence of embeddings, one for each patch of the input images.
            # If you're looking for a single vector representation per image, typically the class token embedding (the first token) is used. 
            # If the model doesn't use a class token, you might need to apply a different pooling strategy over the patch embeddings.
        
        ## Updating to reshape
        
        # Before reshaping, x is in shape [batch_size, num_patches+1, embedding_dim]
        # We discard the first token which is used for classification in the original ViT model
        x = x[:, 1:, :]  # Now in shape [batch_size, num_patches, embedding_dim]
        
        # Reshape to [batch_size, num_patches_side, num_patches_side, embedding_dim]
        x = x.reshape(-1, self.num_patches_side, self.num_patches_side, x.size(-1))

        # Permute to get [batch_size, embedding_dim, num_patches_side, num_patches_side]
        # This is a pseudo-spatial 2D grid, where embedding_dim becomes the channel dimension
        x = x.permute(0, 3, 1, 2)
        
        return x



In [3]:
## Adding in CNN component


## CNN for regression
class RegressionCNN(nn.Module):
    ## Maybe try messing w/ kernel_size, padding and stride?
    def __init__(self, embedding_dim):
        super(RegressionCNN, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=embedding_dim, out_channels=256, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.sig1 = nn.Sigmoid()

        # Second convolutional layer
            ## Same amount? Dropout should help it to learn other things
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.sig2 = nn.Sigmoid()
        
        # Third convolutional layer -- new
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.sig3 = nn.Sigmoid()
        
        
        # Implementing dropout
        self.dropout = nn.Dropout(p=0.1)

        # Adaptive pooling layer to pool down to 1x1
        self.adapt_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Final fully connected layer for regressin
        self.fc = nn.Linear(in_features=64, out_features=1)

    def forward(self, x):
        # Apply first convolution, batch norm, and activation
        x = self.sig1(self.bn1(self.conv1(x)))
        
        # Apply second convolution, batch norm, and activation
        x = self.sig2(self.bn2(self.conv2(x)))
        
        # Apply third convolution, batch norm, and activation -- new
        x = self.sig3(self.bn3(self.conv3(x)))
        
        # Apply dropout
        x = self.dropout(x)

        # Pool the output from the convolutional layers
        x = self.adapt_pool(x)

        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layer for regression
        x = self.fc(x)
        return x

def calculate_rmse_and_r2(loader, model, scaler):
    model.eval()
    targets, predictions = [], []
    with torch.no_grad():
        for batch in loader:
            images, labels = batch
            embeddings = custom_model(images)  # Get embeddings from the ViT
            preds = cnn_model(embeddings)  # Pass embeddings to the CNN
            predictions.extend(preds.view(-1).tolist())
            targets.extend(labels.tolist())

    # Scale the targets using the provided scaler
    targets_scaled = scaler.transform(np.array(targets).reshape(-1, 1)).flatten()

    # Convert to tensors
    predictions = torch.tensor(predictions)
    targets_scaled = torch.tensor(targets_scaled)

    # Calculate RMSE on scaled targets
    rmse_value = torch.sqrt(nn.functional.mse_loss(predictions, targets_scaled))

    # Calculate R^2 on scaled targets
    r2_value = r2_score(targets_scaled, predictions)

    return rmse_value.item(), r2_value


In [8]:
def evaluate_model(model, loader, custom_model, scaler, return_depths=False, return_source = False):
    model.eval()
    predictions, actuals, depths,sources = [], [], [], []
    with torch.no_grad():
        for batch in loader:
            indices, images, labels, batch_depths, batch_source = batch
            embeddings = custom_model(images)
            preds = model(embeddings)
            predictions.extend(preds.view(-1).cpu().numpy())
            actuals.extend(labels.cpu().numpy())
            if return_depths:
                depths.extend(batch_depths.cpu().numpy())
            if return_source:
                sources.extend(batch_source)

    # Scale back if necessary
    if scaler:
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
        actuals = scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

    return sources, depths, actuals, predictions



In [5]:

def plot_results(train_depths, train_actuals, train_predictions, train_sources, test_depths, test_actuals, test_predictions, test_sources, title, save_path):
    plt.figure(figsize=(10, 9))

    # Convert to numpy arrays if not already
    train_depths = np.array(train_depths)
    train_actuals = np.array(train_actuals)
    train_sources = np.array(train_sources)
    train_predictions = np.array(train_predictions)
    test_depths = np.array(test_depths)
    test_actuals = np.array(test_actuals)
    test_sources = np.array(test_sources)
    test_predictions = np.array(test_predictions)

    # Create a grid with 2 rows and 1 column, 
    # the first row is twice the height of the second row
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 

    # Main plot for actual values
    ax1 = plt.subplot(gs[0])

    for lake in ['LVID', 'SVID']:
        # Filter train and test data
        train_mask = (train_sources == lake)
        test_mask = (test_sources == lake)
        
        #print(f"{lake} - Train mask sum: {np.sum(train_mask)}, Test mask sum: {np.sum(test_mask)}")

        # Check and concatenate
        if np.any(train_mask) or np.any(test_mask):
            all_depths = np.concatenate([train_depths[train_mask], test_depths[test_mask]])
            all_actuals = np.concatenate([train_actuals[train_mask], test_actuals[test_mask]])
            #print("Check\n", all_depths, "\n", all_actuals)
            sorted_indices = np.argsort(all_depths)
            plt.plot(all_depths[sorted_indices], all_actuals[sorted_indices], label=f'{lake} Actual', linestyle='-', marker='o')

    ax1.set_xlabel('Sediment Depth')
    ax1.set_ylabel('Target Variable')
    ax1.set_title(title)
    ax1.legend()
    ax1.grid(True)

    # Subplot for residuals
    ax2 = plt.subplot(gs[1])

    for lake in ['LVID', 'SVID']:
        # Filter train and test data for residuals
        train_mask = (train_sources == lake)
        test_mask = (test_sources == lake)

        # Check and plot residuals
        if np.any(train_mask):
            train_depths_lake = train_depths[train_mask]
            train_residuals = abs(train_actuals[train_mask] - train_predictions[train_mask])
            sorted_indices = np.argsort(train_depths_lake)
            plt.plot(train_depths_lake[sorted_indices], train_residuals[sorted_indices], label=f'{lake} Train Residual', linestyle='-', marker='x')

        if np.any(test_mask):
            test_depths_lake = test_depths[test_mask]
            test_residuals = abs(test_actuals[test_mask] - test_predictions[test_mask])
            sorted_indices = np.argsort(test_depths_lake)
            plt.plot(test_depths_lake[sorted_indices], test_residuals[sorted_indices], label=f'{lake} Test Residual', linestyle='--', marker='o')

    ax2.set_xlabel('Sediment Depth')
    ax2.set_ylabel('Residuals')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()


def old_plot_results(train_depths, train_actuals, train_predictions, train_sources, test_depths, test_actuals, test_predictions, test_sources, title, save_path):
    
    plt.figure(figsize=(10, 6))

    # Convert sources to numpy arrays if they are not already
    train_depths = np.array(train_depths)
    train_actuals = np.array(train_actuals)
    train_sources = np.array(train_sources)
    train_predictions = np.array(train_predictions)
    test_depths = np.array(test_depths)
    test_actuals = np.array(test_actuals)
    test_predictions = np.array(test_predictions)
    test_sources = np.array(test_sources)

    check = 0
    for lake in ['LVID', 'SVID']:
        # Filter and check train data
        lake_train_mask = train_sources == lake
        if np.any(lake_train_mask):
            lake_train_depths = train_depths[lake_train_mask]
            lake_train_actuals = train_actuals[lake_train_mask]
            sorted_train_indices = np.argsort(lake_train_depths)
            plt.plot(lake_train_depths[sorted_train_indices], lake_train_actuals[sorted_train_indices], label=f'{lake} Train', linestyle='-', marker='o')
            check += 1

        # Filter and check test data
        lake_test_mask = test_sources == lake
        if np.any(lake_test_mask):
            lake_test_depths = test_depths[lake_test_mask]
            lake_test_actuals = test_actuals[lake_test_mask]
            lake_test_predictions = test_predictions[lake_test_mask]
            sorted_test_indices = np.argsort(lake_test_depths)
            plt.plot(lake_test_depths[sorted_test_indices], lake_test_actuals[sorted_test_indices], label=f'{lake} Test Actual', linestyle='-', marker='o')
            plt.plot(lake_test_depths[sorted_test_indices], lake_test_predictions[sorted_test_indices], label=f'{lake} Test Predicted', linestyle='--', marker='x')
            check += 1
            
    if check == 0:
        return "No data to plot!"
    
    plt.xlabel('Sediment Depth')
    plt.ylabel('Target Variable')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.show()    
    

In [11]:
def run_experiment(train_loader, test_loader, cnn_model, custom_model, scaler, num_epochs=100, lr=0.001):
    optimizer = optim.Adam(cnn_model.parameters(), lr=lr)
    loss_function = nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        # Training steps...
        cnn_model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            images, labels = batch[1], batch[2]
            print(images.shape)
            embeddings = custom_model(images)
            predictions = cnn_model(embeddings)
            loss = loss_function(predictions.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

    # Evaluate on test data
    train_sources, train_depths, train_actuals, train_predictions = evaluate_model(cnn_model, train_loader, custom_model, scaler, return_depths=True, return_source = True)
    test_sources, test_depths, test_actuals, test_predictions = evaluate_model(cnn_model, test_loader, custom_model, scaler, return_depths=True, return_source = True)

    return train_depths, train_actuals, train_sources, train_predictions, test_depths, test_actuals, test_predictions, test_sources

In [12]:
results = []

## Define the combinations
combinations = [
    #("lvid", "lvid"),
    #("lvid", "svid"),
    #("lvid", "both"),
    #("svid", "lvid"),
    ("svid", "svid"),
    #("svid", "both"),
    #("both", "lvid"),
    #("both", "svid"),
    #("both", "both")
]
targets = ["%toc"]
# Load the pre-trained ViT model
pretrained_vit = ViTForImageClassification.from_pretrained('facebook/deit-tiny-patch16-224')

## Freeze params
for param in pretrained_vit.parameters():
    param.requires_grad = False

# Create model w first three layers and create embedding
custom_model = CustomViTEmbeddingModel(pretrained_vit)

for target in targets:
    for train_set, test_set in combinations:
        cnn_model = RegressionCNN(embedding_dim=192)
        print(f"Running experiment for Target: {target}, Train: {train_set}, Test: {test_set}")
        
        # Load data
        train_loader, _, train_scaler = load_data(target, lake=train_set, set="full", scaled=False, sediment_width = 1)
        _, test_loader, _ = load_data(target, lake=test_set, set="full", scaled=False, sediment_width = 1)
        
        train_depths, train_actuals, train_sources, train_predictions, test_depths, test_actuals, test_predictions, test_sources = run_experiment(train_loader, test_loader, cnn_model, custom_model, train_scaler, num_epochs=100, lr=0.001)
        #print(train_sources,test_sources)
        
        timestamp = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")
        save_dir = 'data_exp/1cm/plots'
        os.makedirs(save_dir, exist_ok=True)
        filename = f'{target}_{train_set}train{test_set}_test_{timestamp}.png'
        plot_title = f"{target} Predictions (Train: {train_set}, Test: {test_set})"
        save_path = os.path.join(save_dir, filename)
        plot_results(train_depths, train_actuals, train_predictions, train_sources,  test_depths, test_actuals, test_predictions, test_sources, plot_title, save_path)
        filename = f'OLD{target}_{train_set}train{test_set}_test_{timestamp}.png'
        save_path = os.path.join(save_dir, filename)
        old_plot_results(train_depths, train_actuals, train_predictions, train_sources, test_depths, test_actuals, test_predictions, test_sources, plot_title, save_path)
        # Calculate performance metrics
        rmse_value = np.sqrt(mean_squared_error(test_actuals, test_predictions))

        # Add the results to the results dataframe
        results.append({
            "Train Set": train_set,
            "Test Set": test_set,
            "Target": target,
            "RMSE": rmse_value
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

timestamp = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")
save_dir = 'data_exp/1cm/results'

# Saving the plot
# Ensure the target directory exists
os.makedirs(save_dir, exist_ok=True)
filename = f'experiment_results_{timestamp}.csv'


# Save the results to a CSV file
results_df.to_csv(os.path.join(save_dir, filename), index=False)

Running experiment for Target: %toc, Train: svid, Test: svid
%toc
(<torch.utils.data.dataloader.DataLoader object at 0x3238f5650>, <torch.utils.data.dataloader.DataLoader object at 0x323940390>, None)
tensor([[[[ 85.,  91.,  81.],
          [ 85.,  88.,  79.],
          [ 87.,  90.,  81.],
          ...,
          [ 83.,  88.,  81.],
          [ 86.,  91.,  84.],
          [ 82.,  87.,  80.]],

         [[ 86.,  92.,  82.],
          [ 86.,  89.,  80.],
          [ 88.,  91.,  82.],
          ...,
          [ 85.,  90.,  83.],
          [ 83.,  88.,  81.],
          [ 87.,  92.,  85.]],

         [[ 84.,  89.,  82.],
          [ 84.,  87.,  80.],
          [ 85.,  88.,  81.],
          ...,
          [ 85.,  90.,  83.],
          [ 87.,  92.,  85.],
          [ 83.,  88.,  81.]],

         ...,

         [[ 13.,  12.,   8.],
          [ 17.,  16.,  12.],
          [ 16.,  15.,  11.],
          ...,
          [ 16.,  15.,  11.],
          [ 16.,  15.,  11.],
          [ 15.,  14.,  10.]

ValueError: Make sure that the channel dimension of the pixel values match with the one set in the configuration. Expected 3 but got 1300.

In [ ]:
!pwd
!ls "/Users/willhoff/Desktop/thesis_2024/data/img_data/SVID"